In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def get_train_image_path(folder, path='./input/data/train/images/'):
    file_path_list = glob(path + folder + '/*')
    return file_path_list


def figure_out_mask_label(file_path):
    file_name = file_path.split('/')[-1]
    if 'incorrect' in file_name: return 'incorrect'
    elif 'mask' in file_name: return 'wear'
    else: return 'not_wear'


def get_label(mask, gender, age):
    if mask == 'wear' and gender == 'male' and age < 30:
        return 0
    if mask == 'wear' and gender == 'male' and (age >= 30  and age < 58):
        return 1
    if mask == 'wear' and gender == 'male' and age >= 58:
        return 2
    if mask == 'wear' and gender == 'female' and age < 30:
        return 3
    if mask == 'wear' and gender == 'female' and (age >= 30  and age < 58):
        return 4
    if mask == 'wear' and gender == 'female' and age >= 58:
        return 5
    if mask == 'incorrect' and gender == 'male' and age < 30:
        return 6
    if mask == 'incorrect' and gender == 'male' and (age >= 30  and age < 58):
        return 7
    if mask == 'incorrect' and gender == 'male' and age >= 58:
        return 8
    if mask == 'incorrect' and gender == 'female' and age < 30:
        return 9
    if mask == 'incorrect' and gender == 'female' and (age >= 30  and age < 58):
        return 10
    if mask == 'incorrect' and gender == 'female' and age >= 58:
        return 11
    if mask == 'not_wear' and gender == 'male' and age < 30:
        return 12
    if mask == 'not_wear' and gender == 'male' and (age >= 30  and age < 58):
        return 13
    if mask == 'not_wear' and gender == 'male' and age >= 58:
        return 14
    if mask == 'not_wear' and gender == 'female' and age < 30:
        return 15
    if mask == 'not_wear' and gender == 'female' and (age >= 30  and age < 58):
        return 16
    if mask == 'not_wear' and gender == 'female' and age >= 58:
        return 17
    
    
def divide_train_valid_df(df):
    train_x, valid_x, train_y, valid_y = train_test_split(df[['gender', 'full_path','age','mask']], 
                                                      df['label'], 
                                                      test_size = 0.2, 
                                                      random_state = 42, 
                                                      stratify = df['label'])
    train_df = pd.merge(train_x, train_y, how='inner', left_index=True, right_index=True)
    valid_df = pd.merge(valid_x, valid_y, how='inner', left_index=True, right_index=True)
    return train_df, valid_df

In [3]:
train_df = pd.read_csv('./input/data/train/train.csv')
submission_df = pd.read_csv('./input/data/eval/info.csv')

In [4]:
# 각 폴더 내에 있는 파일 경로 읽어오기
train_df['path_list'] = train_df['path'].progress_apply(get_train_image_path)

100%|██████████| 2700/2700 [00:00<00:00, 20277.90it/s]


In [5]:
# 리스트화된 컬럼을 gender, age에 맞게 펼쳐준 뒤, merge하여 새로운 df 생성
gender_df = pd.DataFrame({'gender':np.repeat(train_df['gender'].values, train_df['path_list'].str.len()),
                          'full_path':np.concatenate(train_df['path_list'].values)
                         })

age_df = pd.DataFrame({'age':np.repeat(train_df['age'].values, train_df['path_list'].str.len()),
                       'full_path':np.concatenate(train_df['path_list'].values)
                      })

new_df = pd.merge(gender_df, age_df, how='inner', on='full_path')

In [6]:
# mask 여부 확인
new_df['mask'] = new_df['full_path'].progress_apply(figure_out_mask_label)

100%|██████████| 18900/18900 [00:00<00:00, 497613.67it/s]


In [7]:
# label 생성
new_df['label'] = new_df[['mask', 'gender', 'age']].progress_apply(lambda x: get_label(x[0], x[1], x[2]), axis=1)

100%|██████████| 18900/18900 [00:00<00:00, 68537.16it/s]


In [9]:
new_df['label'].value_counts()

3     3660
4     3345
0     2745
1     1570
5     1285
2      895
15     732
9      732
10     669
16     669
12     549
6      549
7      314
13     314
11     257
17     257
8      179
14     179
Name: label, dtype: int64

In [10]:
# 새로운 DataFrame 저장
new_df.to_csv('./intergrated_train.csv', index=False)

In [ ]:
new_df = pd.read_csv('./intergrated_train.csv')
new_df.head()

In [ ]:
train_df, valid_df = divide_train_valid_df(new_df)
train_df.to_csv('./train_df.csv', index=False)
valid_df.to_csv('./valid_df.csv', index=False)
print(len(train_df))
print(len(valid_df))